In [1]:
import os
import pandas as pd
import csv
import requests

In [ ]:
if os.path.exists("Footfall Data/glasgow_springboard.csv"):
    glasgow_springboard = pd.read_csv("Footfall Data/glasgow_springboard.csv")
else:
    r = get_report("https://api.glasgow.gov.uk/mobility/v1/footfall/historical?format=json&startDate=2010-01-01&endDate=2022-01-01")
    report_parsed = json.loads(r)
    glasgow_springboard = pd.DataFrame()
    for i in report_parsed:
        this_line = pd.DataFrame({'Timestamp':i['processDate'], 'Count': i['pedestrianCount'],
                           'latitude': i['location']['latitude'], 'longitude': i['location']['longitude'],
                            'name': i['location']['description']},index = [0])
        glasgow_springboard = pd.concat([glasgow_springboard, this_line], axis = 0)
#     glasgow_springboard.to_csv("glasgow_springboard.csv")

In [76]:
CSV_URL ="https://api.glasgow.gov.uk/mobility/v1/footfall/historical?format=csv&startDate=2005-01-01&endDate=2022-01-01"

with requests.Session() as s:
    download = s.get(CSV_URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

glasgow_springboard = pd.DataFrame(my_list)
glasgow_springboard.columns = glasgow_springboard.iloc[0]
glasgow_springboard = glasgow_springboard[1:]

glasgow_springboard[" locationLatitude"] = pd.to_numeric(glasgow_springboard[" locationLatitude"])
glasgow_springboard[" locationLongitude"] = pd.to_numeric(glasgow_springboard[" locationLongitude"])

len(glasgow_springboard)
# glasgow_springboard.to_csv("Footfall Data/glasgow_counts_fromcsvapi.csv", index=False) 

220128

In [57]:
# for 
for sensor_name in glasgow_springboard[' locationDescription'].unique():
    print(sensor_name)
    one_sensor = glasgow_springboard[glasgow_springboard[' locationDescription'] == sensor_name]
    print(one_sensor[' processDate'].min())

Argyll Street near Queen Street Junction
2018-02-23T00:00:00.0000000Z
Buchanan Street near Buchanan Galleries Entrance
2018-02-23T00:00:00.0000000Z
Buchanan Street near Gordon Street Junction
2018-02-23T00:00:00.0000000Z
Sauchiehall Street near Wellington Street Junction
2018-02-23T00:00:00.0000000Z
194 Bath Street (near Blythswood Street Junction)
2018-02-23T00:00:00.0000000Z
253 High Street (near George Street Junction)
2020-01-15T00:00:00.0000000Z
Sauchiehall Street near Elmbank Street Junction
2018-02-23T00:00:00.0000000Z


In [ ]:
glasgow_springboard_map = folium.Map(location=[glasgow_springboard[" locationLatitude"].mean(),
                           glasgow_springboard[" locationLongitude"].mean()], zoom_start=12, control_scale=True, min_zoom = 12)
for index, location_info in glasgow_springboard.iterrows():
    folium.Marker([location_info[" locationLatitude"], location_info[" locationLongitude"]], popup=location_info[" locationDescription"]).add_to(glasgow_springboard_map)
glasgow_springboard_map

In [158]:
# Get data for just one camera
one_camera = glasgow_cctv[glasgow_cctv['cameraName'] == 'C94'].copy()
# CReate a timestamp column
one_camera['Datetime'] = pd.to_datetime(one_camera['Timestamp'])
# Reset index
# one_camera.reset_index(drop = True, inplace = True)
one_camera = one_camera[['Datetime', 'person']]
one_camera.reset_index(drop=True, inplace = True)
one_camera 
# # Get the individual days/months/years
# one_camera['day'] = one_camera['Datetime'].dt.day
# one_camera['month'] = one_camera['Datetime'].dt.month
# one_camera['year'] = one_camera['Datetime'].dt.year
# one_camera['hour'] = one_camera['Datetime'].dt.hour

# This suggests that there are not days missing
# https://stackoverflow.com/questions/47231496/pandas-fill-missing-dates-in-time-series
len(one_camera)
idx = pd.date_range(min(one_camera['Datetime']), max(one_camera.Datetime))

# Add in missing dates with a NaN values
one_camera.set_index('Datetime', inplace = True)
print("len idx: ", len(idx), "len data:" ,len(one_camera))
test = one_camera.reindex(idx, fill_value=np.nan)
test[test['person'].isna()]


len idx:  280 len data: 278


,person
2021-11-03,NaN
2022-01-25,NaN
